Importing Neccesary Library

In [2]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

Loading ENV File

In [5]:
load_dotenv(override = True)

api_key = os.getenv('venice')

MODEL = 'cognitivecomputations/dolphin-mistral-24b-venice-edition:free'

In [7]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= api_key,
)

Web Scraping

In [10]:

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

Get Usable Links using LLM

In [13]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [21]:
def get_links(url):
    website = Website(url)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [23]:
get_links("https://www.geeksforgeeks.org/")

{'links': [{'type': 'about page',
   'url': 'https://www.geeksforgeeks.org/about/'},
  {'type': 'jobs and careers page',
   'url': 'https://geeksforgeeks.zohorecruit.in/careers'},
  {'type': 'community page', 'url': 'https://connect.geeksforgeeks.org/'}]}

In [25]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'enterprise services page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join us page', 'url': 'https://huggingface.co/join'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'about us page', 'url': 'https://huggingface.co/brand'},
  {'type': 'social media - twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media - linkedin',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

Make The Brochure

In [30]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [32]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [36]:
def create_brochure(company_name, url):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [38]:
create_brochure("GeeksForGeeks", "https://www.geeksforgeeks.org/")

# Welcome to GeeksforGeeks: Where Nerds Unleash Their Potential!

## About Us

Ever wondered where nerds go to become super nerds? Welcome to GeeksforGeeks, your ultimate learning playground! We're not just a website; we're a thriving community of tech enthusiasts, where geeks come to geek out!

### Our Mission

To make learning tech as fun as leveling up in your favorite video game. We believe that every geek deserves to learn, grow, and dominate the tech world with their skills!

## What We Offer

### Courses

From Python to DevOps, we've got courses that'll make you say, "I can't believe I'm actually enjoying this!" Here are a few of our fan favorites:

1. **DSA to Development: A Complete Guide** - For those who want to go from zero to hero in Data Structures and Algorithms.
2. **JAVA Backend Development - Live** - Because who doesn't love a good live coding session?
3. **Full Stack Development with React & Node JS** - Turn your ideas into reality with this project-based training.
4. **Tech Interview 101** - From DSA to System Design, we've got you covered for your dream tech job.

### Tutorials and Practice

- **Practice Problems** - Expand your mind with puzzles and challenges in C, C++, Java, Python, and more!
- **Interview Preparation** - Got an interview? We've got your back with company-specific prep and aptitude tests.

### Community

Join our community and become part of a network where you can share your experiences, learn from others, and maybe even find your tech soulmate! We've got:

- Personalized feed with the latest tech trends and discussions.
- Interview experiences shared by real geeks, like you.
- Job and internship opportunities tailored just for you.

## Why Join GeeksforGeeks?

- **Learn with Peers** - Because sharing is caring, especially when it comes to code!
- **Build Your Skills** - From scratch to star, we'll take you on a journey of continuous learning.
- **Visibility and Career Growth** - Showcase your projects, build a portfolio, and let the world know you're a tech superstar!

## Careers at GeeksforGeeks

Looking to join our merry band of tech enthusiasts? We're always on the lookout for passionate geeks to help us grow and innovate. From interns to full-time roles, there's something for everyone at GeeksforGeeks!

## Contact Us

**Corporate Office:**
A-143, 7th Floor, Sovereign Corporate Tower, Sector- 136, Noida, Uttar Pradesh (201305)

**Registered Office:**
K 061, Tower K, Gulshan Vivante Apartment, Sector 137, Noida, Gautam Buddh Nagar, Uttar Pradesh, 201305

Have a question or just want to say hi? Reach out to us at [contact details].

## Join the Geek Revolution!

Ready to geek out and level up your skills? Join GeeksforGeeks today and let the journey begin! 🚀

*Nerds of the world, unify!*

In [44]:
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [46]:
stream_brochure("GeeksForGeeks", "https://www.geeksforgeeks.org/")

# GeeksForGeeks: Your Ultimate Learning Hub!

## Welcome to the World of Geeks!

Ever felt like a fish out of water in the tech world? Well, GeeksForGeeks is here to turn you into a tech-savvy shark! We're your one-stop shop for all things coding, offering a smorgasbord of courses, tutorials, and practice problems that'll make your brain tickle with joy.

## What's Cooking at GeeksForGeeks?

### Courses Galore!
- **Data Structures & Algorithms (DSA):** Because who doesn't love a good puzzle?
- **Web Development:** From HTML to JavaScript, we've got you covered like a well-designed webpage.
- **Data Science & Machine Learning:** Dive into the world of AI and come out as a восстаниан на машинното учение.
- **Linux & DevOps:** Become a system ninja with our top-notch courses.

### Practice Makes Perfect
- **Coding Problems:** Solve your way to glory with hundreds of practice problems.
- **Interview Preparation:** Ace those tech interviews with our expert tips and tricks.

### Community Vibes
Join our thriving peer network and grow together. Share interview experiences, job openings, and more. Who said learning can't be social?

## Why Choose GeeksForGeeks?

### Exclusive Perks
- **Personalized Feed:** Get content tailored just for you. Because one size fits all is so last season.
- **Job Openings & Updates:** Stay in the loop with the latest job opportunities. Your dream job is just a click away!

### Success Stories
Hear it from our happy geeks:
- "On-campus Interview Experience for SDE" — turned a nerd into a CTO!
- "Web3 is trending because it’s changing the internet as we know it!" — said the geek who became a blockchain guru.

## Careers at GeeksForGeeks

Looking for a job where you can geek out and get paid for it? Join our team and be part of a company that loves tech as much as you do. Check out our [Careers](https://www.geeksforgeeks.org/careers/) page for open positions and become the next big thing in the tech world!

## Join the Geek Revolution!

Whether you're a student, a professional, or just someone who loves to tinker with code, GeeksForGeeks is your home. Come geek out with us and let's build something amazing together!

*GeeksForGeeks — Where techies become rock stars!* 🎓💻🚀